In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [4]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

## (TEST) Checking actual delay (from the start of the 3 epochs)

In [16]:
df_cumdelay = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d450_a0.1_b0.8_c0.1.pkl')
df_delay0 = df_cumdelay.loc[60, :]

df_instdelay = df_cumdelay.diff()
df_instdelay.loc[60, :] = df_delay0
df_instdelay

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,22.493678,0.000000,20.537077,0.000000,21.114231,94.934741,4.845692,39.787077,4.117124,8.210244,...,14.721174,36.925418,19.232748,18.409559,18.116802,45.033541,33.538898,43.928847,31.659987,79.609408
120,105.737355,176.065366,187.824153,140.447385,116.471282,131.505358,76.802615,99.574153,37.718495,81.590977,...,39.007715,119.915024,38.219781,57.704458,46.994213,198.243463,74.844514,122.480926,108.470746,392.253965
180,131.859194,283.920489,187.824153,414.993463,253.063973,408.488963,179.068153,284.839691,109.405486,107.551222,...,103.743785,242.617660,68.630090,156.436991,140.424989,329.197385,131.185705,299.736927,282.396807,813.909130
240,247.346550,283.920489,385.111230,414.993463,418.656665,676.880197,268.929383,443.913845,192.741106,115.156344,...,146.294363,406.097407,131.342370,365.604158,411.049400,503.330154,256.945992,646.508465,597.871196,1040.293078
300,443.212066,559.735855,517.552615,550.592156,533.127947,861.929061,367.253536,583.737998,262.693849,163.931955,...,243.046507,706.719996,351.468603,717.348693,558.117445,811.827154,563.482212,1288.760930,904.334976,1388.800364
360,556.833905,694.840977,563.861230,686.940849,536.749357,1036.885555,396.290613,638.216459,258.693849,256.667810,...,436.260930,969.917167,595.055615,1029.621303,825.086905,1064.444921,813.953601,1877.137162,1195.659987,1631.346121
420,513.840227,660.985855,526.361230,681.842156,523.620767,1130.113802,524.673382,615.466459,264.795221,227.167810,...,577.842477,1187.633198,671.364981,1411.232426,915.128268,1216.040613,997.380090,2318.198778,1397.474389,1635.846121
480,682.705744,553.130733,526.361230,407.296078,503.992177,1156.092049,629.614766,504.775075,286.928097,217.667810,...,754.542970,1077.571260,928.241213,1562.269181,1126.427025,1102.500536,1147.634912,2483.347085,1581.565902,1262.585512
540,608.212066,708.985855,418.361230,742.092156,451.006537,1240.363802,615.290613,533.025075,252.810973,236.588298,...,1047.262645,1388.646057,1129.386198,2148.721817,1134.653879,1417.071153,1330.624196,3111.275932,1663.086798,1678.958851


In [35]:
list(df_instdelay.sum(axis=1)[[60, 240, 420]])

[586.9112730390056, 8391.820188598369, 19665.289327046718]

## (TEST) Checking expected delay (from the start of 3 epochs)

In [24]:
df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')
df_milp.head(10)

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,174.118326,767.272152,51.278481,382.056221,450,True,0.80,0.10,0.10,Delay priority,0
5,456.304917,767.272152,51.278481,-598.035046,450,True,0.45,0.10,0.45,Delay-Flow priority,0
3,343.920306,767.272152,51.278481,49.987495,450,True,0.45,0.45,0.10,Delay-Throughput priority,0
6,917.078324,767.272152,51.278481,-487.789937,450,True,0.33,0.33,0.33,Equal priority,0
2,891.892373,801.107595,54.000000,-1585.763682,450,True,0.10,0.10,0.80,Flow priority,0
0,0.227479,743.113924,66.835443,66.835443,450,False,0.00,0.00,0.00,Parent model,0
1,415.136684,791.607595,55.000000,-291.456345,450,True,0.10,0.80,0.10,Throughput priority,0
4,596.610098,791.607595,55.000000,-937.523394,450,True,0.10,0.45,0.45,Throughput-Flow priority,0
1,5.594564,2080.658228,60.000000,1209.951345,900,True,0.80,0.10,0.10,Delay priority,0
6,6.865374,2080.658152,60.000003,-643.922642,900,True,0.45,0.10,0.45,Delay-Flow priority,0


In [37]:
df_milp[(df_milp.demand == 450) & (df_milp.model_type == 'Throughput priority')].delay

[791.6075949367089, 4326.65822785, 6555.15189873]

## Combining expected and actual results

In [5]:
_weights = [
    (0.8, 0.1, 0.1),
    (0.1, 0.8, 0.1),
    (0.1, 0.1, 0.8),
    (0.45, 0.45, 0.1),
    (0.1, 0.45, 0.45),
    (0.45, 0.1, 0.45),
    (0.33, 0.33, 0.33),
    (0, 0, 0),
]

_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_model_type = {
    (0.8, 0.1, 0.1): 'Delay priority',
    (0.1, 0.8, 0.1): 'Throughput priority',
    (0.1, 0.1, 0.8): 'Flow priority',
    (0.45, 0.45, 0.1): 'Delay-Throughput priority',
    (0.1, 0.45, 0.45): 'Throughput-Flow priority',
    (0.45, 0.1, 0.45): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [6]:
delay_dfs = []
throughput_dfs = []

df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')
df_delay = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])
df_thru = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])

for demand in _demands:
    for weights in _weights:

        # Get actual values
        
        df_cumdelay_tmp = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instdelay_tmp = df_cumdelay_tmp.diff()
        df_instdelay_tmp.loc[60, :] = df_cumdelay_tmp.loc[60, :]
        actual_delay = list(df_instdelay_tmp.sum(axis=1)[[60, 240, 420]])
        
        df_cumthru_tmp = pd.read_pickle(DF_PATH + 'cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instthru_tmp = df_cumthru_tmp.diff()
        df_instthru_tmp.loc[60, :] = df_cumthru_tmp.loc[60, :]
        actual_thru = list(df_instthru_tmp.sum(axis=1)[[60, 240, 420]])

        # Get expected values
        
        expected_delay = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].delay)
        expected_thru = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].throughput)
        
        # Combine into a dataframe
        
        for i in range(3):
            df_delay.loc[-1] = [demand, _model_type[weights], expected_delay[i], actual_delay[i], i]
            df_thru.loc[-1] = [demand, _model_type[weights], expected_thru[i], actual_thru[i], i]
            df_delay.index += 1
            df_thru.index += 1

In [7]:
df_delay.to_pickle(DF_PATH + "delay_comparison.pkl")
df_thru.to_pickle(DF_PATH + "throughput_comparison.pkl")